# Analyzing Replication Percentage of Text Generation Using an n-gram Seed and Markov Chains on Sense and Sensibility for Various Levels of n.

Inspired by a local lecture to the Atlanta DSI class given by Caroline Schmitt.  

For exmaple, picking an n-gram at random, randomly pick the next word in the corpus using a Markov chain.  After 75 words are generated, check to see if those 75 words, in order, are present in the corpus.

In [96]:
import pandas as pd
import numpy as np
from collections import defaultdict

from nltk.corpus import gutenberg

## Load Sense and Sensibility as Our Corpus
## Remove punctuation and numbers
## Make all words lower case

In [104]:
punct_num = ['\n', '!', "[", "]",  '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '>', '?']

corpus = gutenberg.raw('austen-sense.txt')

for each in punct_num:
    corpus = corpus.replace(each,' ')

corpus = corpus.lower()
corpus = corpus.split()
normal_text = ' '.join(corpus)

In [106]:
print(set(normal_text))

{'o', 'k', 't', 'x', 'u', 'm', 'n', 'e', ' ', 'j', 'w', 'p', 'f', 'v', 's', 'q', 'c', 'a', 'y', 'b', 'z', 'l', 'g', 'h', 'i', 'd', 'r'}


## Make-pairs generator function:

In [107]:
def make_pairs(corpus):
    for i in range(len(corpus)):
        yield (corpus[i-3]+' '+corpus[i-2], corpus[i-1]+' '+corpus[i])

# note the initial values are negative which will cause the text to wrap upon itself.
# ie, the last two words "the end" will be paired with the first two words "sense and"

In [108]:
pairs = make_pairs(corpus)

## Generate word dictionary:

Create a dictionary such that the keys are each n-gram in the corpus and the word that follows is placed in a list of values.  This dictionary will then be used to generate text by looking up the key and then randomly finding the next two words that follow.

In [109]:
word_dict = {}

for word1, word2 in pairs:
    if word1 in word_dict.keys():
        word_dict[word1].append(word2)
    else:
        word_dict[word1] = [word2]


## Generate new text:

In [137]:
n_iter = 5        # change this to change the length of the markov chain

n = 0

for j in range(10000):
#    first_word_position = 30 
    first_word_position = np.random.choice(len(corpus))

    first_ngram = corpus[first_word_position]+ ' ' + corpus[first_word_position+1]   
    chain = [first_ngram]

    for i in range(n_iter):
        most_recent_word = chain[-1]
        next_ngram = np.random.choice(word_dict[most_recent_word])
        chain.append(next_ngram)


    extract = ' '.join(chain)
    if extract in normal_text:
        n+=1
print(n, n/10000)

366 0.0366


### Observed values of n (ie, probability that the text will self-generate) with various sizes of Markov chains

>  Keeping starting point at word 30

    > n / n_iter / x_bar  
    > 2 / 10000 / 100%  
    > 3 / 10000 / 100%  
    > 4 / 10000 / 100%  
    > 5 / 221 / 2.11%  
    > 6 / 49 / 0.49%  
    > 7 / 17 / 0.17%  
    > 8 / 1 / 0.01%  
    > 9 / 0 / 0.00%  

> And keeping the length of the chain at 5 but selecting the starting 2-gram at random:  
    > 5 / 434 / 4.34%  
    > 5 / 385 / 3.85%  
    > 5 / 372 / 3.72%  
    > 5 / 389 / 3.89%  
    > 5 / 366 / 3.66%

### I was going to quit with the work above but the results of the random starting begged for one bit of analysis.  What does the mean and standard deviation look like for a random starting point and Markov chain of length 5?

In [145]:
#  This takes quite a while to run....

'''
successes = []
for k in range(200):
    print(k)
    n_iter = 5        # change this to change the length of the markov chain
    n = 0

    for j in range(10000):
    #    first_word_position = 30 
        first_word_position = np.random.choice(len(corpus))-1

        first_ngram = corpus[first_word_position]+ ' ' + corpus[first_word_position+1]   
        chain = [first_ngram]

        for i in range(n_iter):
            most_recent_word = chain[-1]
            next_ngram = np.random.choice(word_dict[most_recent_word])
            chain.append(next_ngram)


        extract = ' '.join(chain)
        if extract in normal_text:
            n+=1
    successes.append(n)
'''

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [147]:
np.mean(successes)

389.425

In [148]:
np.std(successes)

18.512276332207232